In [ ]:
#Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import time
from scipy.stats import linregress
import scipy.stats as stats
from config import api_key



In [ ]:
#Read data
game_stats = pd.read_csv("C:/Users/dreww/Desktop/DataAnalysisWork/Project 1/stats.csv")
game_results = pd.read_csv("C:/Users/dreww/Desktop/DataAnalysisWork/Project 1/results.csv")

Home vs Away Wins 

In [ ]:
#Calculate total home and away wins
wins_distrubution = pd.DataFrame(columns=['team', 'home wins', 'away wins'])
num_games = game_results['result'].count()
home_team = game_results['home_team'].unique()
away_team = game_results['away_team'].unique()


home_wins_count = {}
away_wins_count = {}

# Iterate through the DataFrame
for index, row in game_results.iterrows():
    home_team, away_team, result = row['home_team'], row['away_team'], row['result']
    
    # Check if it's a home win
    if result == 'H':
        if home_team in home_wins_count:
            home_wins_count[home_team] += 1
        else:
            home_wins_count[home_team] = 1
    
    # Check if it's an away win
    elif result == 'A':
        if away_team in away_wins_count:
            away_wins_count[away_team] += 1
        else:
            away_wins_count[away_team] = 1

# Print the counts
print("Home Wins:")
for team, count in home_wins_count.items():
    print(f"{team}: {count}")

print("\nAway Wins:")
for team, count in away_wins_count.items():
    print(f"{team}: {count}")

    



In [ ]:
#Graph teams Home wins vs teams away wins
# Create DataFrames from the dictionaries
home_wins_df = pd.DataFrame(list(home_wins_count.items()), columns=['team', 'home wins'])
away_wins_df = pd.DataFrame(list(away_wins_count.items()), columns=['team', 'away wins'])

# Merge the DataFrames based on the 'team' column
merged_df = home_wins_df.merge(away_wins_df, on='team', how='outer').fillna(0)

# Set the width of the bars
bar_width = 0.35
index = range(len(merged_df))

# Create a bar chart to show home and away wins for each team separately
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

# Plot home wins
plt.bar(index, merged_df['home wins'], bar_width, label='Home Wins')

# Plot away wins with an offset
plt.bar([i + bar_width for i in index], merged_df['away wins'], bar_width, label='Away Wins')

plt.xlabel('Team')
plt.ylabel('Wins')
plt.title('Home Wins and Away Wins by Team')
plt.xticks([i + bar_width/2 for i in index], merged_df['team'], rotation=90)
plt.legend()

# Show the chart
plt.show()

In [ ]:
#Test the difference between mean points at home vs mean points away
#First we have to define wins as 3 points, draws as 1 point, and losses as 0 points 
#To do this run a 2 sample t-test comparing results from the 2006-2007 season 
results_2006 = game_results[game_results["season"] == "2006-2007"]
wins_distrubution = pd.DataFrame(columns=['team', 'home points', 'away points','total points'])
num_games = results_2006['result'].count()
home_team = results_2006['home_team'].unique()
away_team = results_2006['away_team'].unique()



# Define a dictionary to store points for each team
team_points = {}

# Iterate through the DataFrame
for index, row in results_2006.iterrows():
    home_team, away_team, result = row['home_team'], row['away_team'], row['result']

    # Initialize points for each team if they don't exist
    if home_team not in team_points:
        team_points[home_team] = 0
    if away_team not in team_points:
        team_points[away_team] = 0

    # Update points based on the result
    if result == 'H':
        team_points[home_team] += 3
    elif result == 'D':
        team_points[home_team] += 1
        team_points[away_team] += 1
    elif result == 'A':
        team_points[away_team] += 3

# Convert the dictionary to a DataFrame
points_df = pd.DataFrame(list(team_points.items()), columns=['team', 'total points'])

# Define dictionaries to store home and away points for each team
home_points = {}
away_points = {}

# Iterate through the DataFrame
for index, row in results_2006.iterrows():
    home_team, away_team, result = row['home_team'], row['away_team'], row['result']

    # Initialize points for each team if they don't exist
    if home_team not in home_points:
        home_points[home_team] = 0
    if away_team not in away_points:
        away_points[away_team] = 0

    # Update points based on the result and location
    if result == 'H':
        home_points[home_team] += 3
    elif result == 'D':
        home_points[home_team] += 1
        away_points[away_team] += 1
    elif result == 'A':
        away_points[away_team] += 3

# Convert the dictionaries to DataFrames
home_points_df = pd.DataFrame(list(home_points.items()), columns=['team', 'home points'])
away_points_df = pd.DataFrame(list(away_points.items()), columns=['team', 'away points'])

# Merge the DataFrames to add home and away points to the existing 'points_df'
merged_points_df = points_df.merge(home_points_df, on='team', how='left').merge(away_points_df, on='team', how='left')
sorted_points_df = merged_points_df.sort_values(by='total points', ascending=False)
print(sorted_points_df)



In [ ]:
#Run paired t-test to determine if there is statistical significance between home and away points
# Perform the paired t-test
t_stat, p_value = stats.ttest_rel(merged_points_df['home points'], merged_points_df['away points'])
print(t_stat)
print(p_value)


Previous Position Ranking

In [ ]:
#Now since we already have a table for position in league in previous season we'll run an analysis for if previous years position effects chances of winning
# Add a new column for position ranking
sorted_points_df['position ranking'] = sorted_points_df['total points'].rank(ascending=False, method='min')

# Sort the DataFrame based on the 'position ranking'
sorted_points_df = sorted_points_df.sort_values(by='position ranking')

points_df_2006 = sorted_points_df


#Now that we have a proper table with positions it's time to show how many points each team gave up in the following season
results_2007 = game_results[game_results["season"] == "2007-2008"]
wins_distrubution = pd.DataFrame(columns=['team', 'home points', 'away points','07/08 points'])
num_games = results_2007['result'].count()
home_team = results_2007['home_team'].unique()
away_team = results_2007['away_team'].unique()



# Define a dictionary to store points for each team
team_points = {}

# Iterate through the DataFrame
for index, row in results_2007.iterrows():
    home_team, away_team, result = row['home_team'], row['away_team'], row['result']

    # Initialize points for each team if they don't exist
    if home_team not in team_points:
        team_points[home_team] = 0
    if away_team not in team_points:
        team_points[away_team] = 0

    # Update points based on the result
    if result == 'H':
        team_points[home_team] += 3
    elif result == 'D':
        team_points[home_team] += 1
        team_points[away_team] += 1
    elif result == 'A':
        team_points[away_team] += 3

# Convert the dictionary to a DataFrame
points_df_2007 = pd.DataFrame(list(team_points.items()), columns=['team', '07/08 points'])
points_df_2007
points_df_2007['07/08 position'] = points_df_2007['07/08 points'].rank(ascending=False, method='min')

# Sort the DataFrame based on the 'position ranking'
points_df_2007 = points_df_2007.sort_values(by='07/08 position')

points_df_2006.rename(columns={'position ranking': '06/07 position'}, inplace=True)
positions_df = points_df_2007.merge(points_df_2006, on='team', how='left')
positions_df.loc[positions_df['team'] == 'Sunderland', '06/07 position'] = 18
positions_df.loc[positions_df['team'] == 'Birmingham City', '06/07 position'] = 19
positions_df.loc[positions_df['team'] == 'Derby County', '06/07 position'] = 20
positions_df.loc[positions_df['team'] == 'Arsenal', '06/07 position'] = 4
positions_df = positions_df.loc[:, ~positions_df.columns.duplicated()]
columns_to_drop = ['home points', 'away points']

positions_df.drop(columns=columns_to_drop, inplace=True)
positions_df




In [ ]:
# Calculate and add the 'points lost' column
max_points = 38 * 3  

positions_df['points lost'] = max_points - positions_df['07/08 points']

#Now create a scatter plot of previous position and points lost for each team
plt.figure(figsize=(10, 6))  


plt.scatter(positions_df['06/07 position'], positions_df['points lost'], c='blue', marker='o', alpha=0.5)
plt.xticks(range(1, len(positions_df) + 1), range(1, len(positions_df) + 1))
plt.xlabel('Previous Position')
plt.ylabel('Points Given Up')
plt.title('Scatter Plot of Previous Position vs. Points Given Up')
plt.grid(True)
plt.show()
    
    

In [ ]:
#Now run a linear regression model to see if there's statistical significance
# Extract the relevant columns
x = positions_df['06/07 position']
y = positions_df['points lost']

# Perform the linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Output the results
print(f"Slope: {slope:.2f}")
print(f"Intercept: {intercept:.2f}")
print(f"R-squared: {r_value**2:.2f}")
print(f"P-value: {p_value:.6f}")
print(f"Standard Error: {std_err:.2f}")

# Create a scatter plot and plot the regression line
plt.scatter(x, y, label='Actual Data')
plt.plot(x, intercept + slope * x, 'r', label='Regression Line')
plt.xlabel('Placement in 06/07 Season')
plt.ylabel('Points Lost in 07/08 Season')
plt.legend()
plt.show()


Win-Streak Section

In [28]:
# Create a new DataFrame to store the results
new_dataframe = pd.DataFrame(columns=['home_team', 'away_team', 'winner'])

# Iterate through the DataFrame
for index, row in game_results.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    result = row['result']

    if result == 'H':
        winner = home_team
    elif result == 'A':
        winner = away_team
    else:
        winner = 'Draw'

    new_entry = {'home_team': home_team, 'away_team': away_team, 'winner': winner}
    new_dataframe = pd.concat([new_dataframe, pd.DataFrame([new_entry])], ignore_index=True)

# Create a new DataFrame to store win streaks
win_streak_df = pd.DataFrame(columns=['team', 'win_streak', '2_game_streaks', '3_game_streaks', '4_game_streaks'])

# Create a dictionary to keep track of win streaks for each team
win_streaks = {}
team_streak_counts = {}

# Iterate through the new_dataframe to update win streaks
for index, row in new_dataframe.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    winner = row['winner']

    # Update the home_team's win streak
    if winner == home_team:
        win_streaks[home_team] = win_streaks.get(home_team, 0) + 1
    else:
        win_streaks[home_team] = 0

    # Update the away_team's win streak
    if winner == away_team:
        win_streaks[away_team] = win_streaks.get(away_team, 0) + 1
    else:
        win_streaks[away_team] = 0

    # Update the streak counts
    for team, streak in win_streaks.items():
        if team not in team_streak_counts:
            team_streak_counts[team] = {'2_game_streaks': 0, '3_game_streaks': 0, '4_game_streaks': 0}

        if streak >= 2:
            team_streak_counts[team]['2_game_streaks'] += 1
        if streak >= 3:
            team_streak_counts[team]['3_game_streaks'] += 1
        if streak >= 4:
            team_streak_counts[team]['4_game_streaks'] += 1

# Append the win streak and streak count information to the win_streak_df using pd.concat
streak_dfs = []
for team, streak in win_streaks.items():
    streak_counts = team_streak_counts[team]
    streak_dfs.append(pd.DataFrame({'team': [team],'2_game_streaks': [streak_counts['2_game_streaks']], '3_game_streaks': [streak_counts['3_game_streaks']], '4_game_streaks': [streak_counts['4_game_streaks']]}))
win_streak_df = pd.concat(streak_dfs, ignore_index=True)

# Display the updated win streak DataFrame
print(win_streak_df)





        


                        team  2_game_streaks  3_game_streaks  4_game_streaks
0           Sheffield United              18               0               0
1                  Liverpool            1158             569             337
2                    Arsenal            1508             859             461
3                Aston Villa             379             130              43
4                    Everton             585             166              50
5                    Watford              97              20               6
6           Newcastle United             390             183              84
7             Wigan Athletic             170              51              14
8                 Portsmouth             118              36               9
9           Blackburn Rovers             194              54               9
10                   Reading             125              49              23
11             Middlesbrough              71              15               0